# Clean : trip statistics

# Purpose
There are some outliers in the trip statistics that will need to be removed.

# Setup

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics

In [ ]:
%reload_kedro
df_stat = catalog.load('uraniborg.trip_statistics_clean')
#df_stat = catalog.load('uraniborg.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True)
df_stat.describe()

In [ ]:
df_stat.head()

In [ ]:
df_stat.shape

In [ ]:
df_stat.info()

In [ ]:
sns.displot(df_stat, x='distance', aspect=3);

In [ ]:
sns.displot(df_stat, x='trip_time', aspect=3);

In [ ]:
df_stat["consumption"] = df_stat["Consumption ME1 (L)"] + df_stat["Consumption ME2 (L)"]

In [ ]:
sns.displot(df_stat, x='consumption', aspect=3);

In [ ]:
df_clean = clean_statistics.clean(df_stat=df_stat, min_distance=4000, min_time=700)

In [ ]:
df_compare = df_stat.copy()
df_compare['item'] = 'raw'

df_clean['item'] = 'clean'
df_compare = df_compare.append(df_clean, ignore_index=True)

In [ ]:
df_compare.describe()

In [ ]:
sns.displot(df_compare, x='E', hue='item', binwidth=5*10**4, aspect=3)

In [ ]:
sns.displot(df_compare, x='trip_time', hue='item', binwidth=25, aspect=3)

In [ ]:
facegrid = sns.relplot(data=df_stat, x='start_time',y='consumption', height=3, aspect=3);
facegrid.set_ylabels('(Fuel consumption per trip) [l]')

In [ ]:
df_ = df_stat.set_index('start_time')
df_.sort_index(inplace=True)

window = "10D"
df_rolling = df_.rolling(window=window).mean().iloc[20:]
df_expanding = df_.expanding().mean().iloc[20:]

facegrid = sns.relplot(data=df_rolling, x='start_time', y='consumption', height=3, aspect=3);
facegrid.set_ylabels('(Fuel consumption per trip) [l]')

In [ ]:
df_.iloc[0].name

In [ ]:
utbildning = '2021-05-10'
df_before = df_.loc[:utbildning]
df_after = df_.loc[utbildning:]


In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)

df_before.plot(y='consumption', label='before', ax=ax, style='b.')
df_after.plot(y='consumption', label='after', ax=ax, style='g.')

df_rolling.plot(y='consumption', label=f'rolling mean ({window})', ax=ax, style='-')
df_expanding.plot(y='consumption', label=f'expanding mean', ax=ax, style='-')
ax.legend();

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)
df_expanding.plot(y='consumption', label=f'expanding mean', ax=ax, style='-')
ax.set_ylim(df_expanding['consumption'].min(),df_before['consumption'].mean())
ax.legend();

In [ ]:
interesting = ['sog','distance', 'consumption']
df_before[interesting].describe()

In [ ]:
df_after[interesting].describe()

In [ ]:
(df_before['consumption'].mean()-df_after['consumption'].mean())/df_before['consumption'].mean()